In [1]:
import requests
import pandas as pd

- One year range limit applies
- Minimum time interval in query response is one day
- Mandatory parameters
- DocumentType
- In_Domain
- Out_Domain
- TimeInterval or combination of PeriodStart and PeriodEnd
- In_Domain and Out_Domain must be populated with the same area EIC code

In [2]:
url_ref = 'https://web-api.tp.entsoe.eu/api?documentType=A44&in_Domain=10YCZ-CEPS-----N&out_Domain=10YCZ-CEPS-----N&periodStart=201512312300&periodEnd=201612312300'

In [11]:
documentType = 'A44'
in_Domain = '10YCZ-CEPS-----N'
out_Domain = '10YCZ-CEPS-----N'
periodStart = '201512312300'
periodEnd = '201612312300'

In [12]:
querystring = {"documentType":documentType,
                "in_Domain":in_Domain,
                "out_Domain":out_Domain,
                "periodStart":periodStart,
                "periodStart":periodEnd}

In [13]:
base_url = 'https://web-api.tp.entsoe.eu/api?'

In [14]:
url = (base_url +
        'documentType=' + documentType +
        '&in_Domain=' + in_Domain + 
        '&out_Domain=' + out_Domain + 
        '&periodStart=' + periodStart + 
        '&periodEnd=' + periodEnd
        )

In [17]:
if url == url_ref:
    print('Same URL!')
else:
    print('Check URL formation')

Same URL!


In [18]:
headers={"Content-Type":"application/xml"}

In [19]:
data = requests.request("GET", url_ref, headers=headers).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data = requests.request("GET", base_url, headers=headers, params=querystring).json()

In [ ]:
data = requests.request("GET", url, headers=headers).json()

In [ ]:
def plot_e_pricing(EIC_code,start_date,end_date,tomorrow_rate):
    # 
    if tomorrow_rate == True:
        now = datetime.datetime.utcnow()
        tomorrow = now + datetime.timedelta(days=1)
        now = now.replace(microsecond=0).isoformat() + 'Z'
        tomorrow = tomorrow.replace(microsecond=0).isoformat() + 'Z'
        # 
        start_date = now
        end_date = tomorrow
    # 
    url = "https://electri.p.rapidapi.com/api/v1/day-ahead-prices"
    querystring = {"to":end_date,"from":start_date,"eic":EIC_code}
    headers = {
      "X-RapidAPI-Key": "7ad267cb53mshf4f5255c38f192ap19f964jsnc7abc0be43eb",
      "X-RapidAPI-Host": "electri.p.rapidapi.com"
    }
    data = requests.request("GET", url, headers=headers, params=querystring).json()
    df = pd.json_normalize(data['data']['series'])
    df['time'] = pd.to_datetime(df['time'])
    # 
    df_metadata = pd.json_normalize(data)
    currency = df_metadata['data.currency'].iloc[0]
    power_unit = df_metadata['data.unit'].iloc[0]
    eic = df_metadata['data.eic'].iloc[0]
    # 
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(df['time'], df['price']);
    plt.title('Electrical prices '+ str(eic))
    plt.xlabel('Time')
    plt.ylabel('Price [' + str(currency) + '/' + str(power_unit) + ']')
    fig.autofmt_xdate()
    # 
    # 
    return fig


inputs = [
    gr.Dropdown(list_of_EIC_codes, label="Electrical Provider EIC"),
    gr.Textbox(label="Start Date [YYYY-MM-DDTHH:MMZ]"),
    gr.Textbox(label="End Date [YYYY-MM-DDTHH:MMZ]"),
    gr.Checkbox(label="Tomorrow's rates?",default='False'),
]
outputs = gr.Plot()

demo = gr.Interface(
    fn=plot_e_pricing,
    inputs=inputs,
    outputs=outputs,
    examples=[
        ["10Y1001A1001A46L","2022-04-05T22:00Z","2022-05-05T22:00Z",False],
        ["10YDK-1--------W","2022-08-05T22:00Z","2022-08-06T22:00Z",False],
        ["10YDK-1--------W"," "," ",True],

    ],
    cache_examples=True,
)

if __name__ == "__main__":
    demo.launch()